`05-rag-conversation.ipynb`

In [ ]:
%pip install -q langchain_pinecone

In [11]:
from dotenv import load_dotenv
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_pinecone import PineconeVectorStore
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from pinecone_utils import create_embeddings, get_vectorstore

load_dotenv()

True

In [7]:
loader = PyMuPDFLoader('./data.pdf')
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
split_docs = text_splitter.split_documents(docs)

index_name = 'gaicc-1'
# 최초 1회만 pdf 를 Pinecone Vectorstore에 저장.
# create_embeddings(index_name=index_name, docs=split_docs)


In [12]:
vectorstore = get_vectorstore(index_name)
retriever = vectorstore.as_retriever()
text = '''
넌 질문-답변을 도와주는 AI 비서야.
아래 제공되는 Context를 통해서 사용자 Question에 대해 답을 해줘야해.

Context에는 직접적으로 없어도, 추론하거나 계산할 수 있는 답변은 최대한 만들어 봐.
그리고 관련 내용을 말할때는, pdf 몇 페이지이서 찾은 내용인지도 말해줘.

답은 적절히 \n를 통해 문단을 나눠줘 한국어로 만들어 줘. 
# Question:
{question}

# Context:
{context}


# Answer:
'''
prompt = PromptTemplate.from_template(text)
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.5)
parser = StrOutputParser()

chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | parser
)

ans = chain.invoke('삼성전자 관련 소식들 다 가져와서 요약해줘')

In [13]:
ans

"삼성전자가 자체 개발한 생성 AI 모델 '삼성 가우스'를 공개했습니다. 삼성 가우스는 언어, 코드, 이미지의 3가지 모델로 구성되어 온디바이스에서 작동 가능하며, 외부로 사용자 정보가 유출될 위험이 없는 안전한 데이터를 통해 학습되었습니다. 또한, 이미지 모델은 창의적인 이미지를 생성하고 기존 이미지를 원하는 대로 변경할 수 있으며, 저해상도 이미지의 고해상도 전환도 지원합니다. 이를 통해 삼성전자는 삼성 가우스를 다양한 제품에 탑재할 계획이며, 온디바이스 AI 기술을 소개하고 있습니다. 또한, 2024년에는 가우스를 탑재한 삼성 스마트폰이 메타의 라마(Llama)2를 탑재한 퀄컴 기기 및 구글 어시스턴트를 적용한 구글 픽셀과 경쟁할 것으로 예상됩니다. 삼성전자의 AI 기술은 산업에서 주목을 받고 있으며, 더 많은 혁신을 기대할 수 있을 것입니다. (출처: data.pdf, page 12)"